In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('data/trainReviews.tsv', sep='\t')

In [3]:
df_train.head()

,id,category,text
0,858,0,burnt money is the perfect festival film it...
1,1762,1,the italian hitchcock and acknowledged mas...
2,235,0,at times you d think edtv would be an entert...
3,712,0,after a marketing windup of striking visuals a...
4,1319,1,john cusack is the kind of actor who seems to ...


In [4]:
#train_review_df.iloc[1]['text']

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [11]:
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
    ])

In [12]:
text_clf.fit(df_train['text'], df_train['category'])

Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])

In [13]:
text_clf['vect'].vocabulary_

{'burnt': 2602,
 'money': 12381,
 'perfect': 13914,
 'festival': 7118,
 'film': 7184,
 'twice': 19901,
 'thankfully': 19224,
 'hear': 8788,
 'seattle': 16820,
 'international': 9976,
 '2001': 120,
 'emerging': 6185,
 'masters': 11806,
 'series': 16968,
 'easily': 5962,
 'year': 21424,
 'worst': 21321,
 'billed': 1919,
 'gay': 7888,
 'bonnie': 2186,
 'clyde': 3522,
 'gritty': 8381,
 'director': 5323,
 'marcelo': 11707,
 'pi': 14077,
 'eyro': 6845,
 'highlight': 8958,
 'designed': 5075,
 'title': 19439,
 'sequence': 16955,
 'lovers': 11403,
 'involved': 10079,
 'bank': 1549,
 'robbery': 16145,
 'makes': 11599,
 'gang': 7829,
 'leader': 10949,
 'plan': 14201,
 'screwed': 16748,
 'angry': 868,
 'causes': 2960,
 'send': 16906,
 'boys': 2290,
 'guys': 8507,
 'actually': 412,
 'hiding': 8951,
 'prostitute': 14864,
 'apartment': 958,
 'men': 12009,
 'fight': 7158,
 'police': 14347,
 'members': 11998,
 'long': 11323,
 'showdown': 17221,
 'movie': 12534,
 'conclusion': 3851,
 'caught': 2955,
 'r

In [16]:
text_clf['tfidf'].idf_

array([6.52146092, 3.88240359, 6.52146092, ..., 6.52146092, 5.82831374,
       6.52146092])

In [17]:
text_clf['clf']

RandomForestClassifier()

In [18]:
df_test = pd.read_csv('data/testReviews.tsv', sep='\t')

In [19]:
test_predicted = text_clf.predict(df_test['text'])

In [20]:
test_predicted

array([1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,

In [21]:
import numpy as np

In [22]:
np.mean(test_predicted == df_test['category'])

0.766

In [23]:
test_predicted_proba = text_clf.predict_proba(df_test['text'])

In [24]:
test_predicted_proba

array([[0.49, 0.51],
       [0.67, 0.33],
       [0.38, 0.62],
       [0.58, 0.42],
       [0.42, 0.58],
       [0.49, 0.51],
       [0.82, 0.18],
       [0.35, 0.65],
       [0.48, 0.52],
       [0.38, 0.62],
       [0.44, 0.56],
       [0.57, 0.43],
       [0.64, 0.36],
       [0.29, 0.71],
       [0.48, 0.52],
       [0.38, 0.62],
       [0.62, 0.38],
       [0.51, 0.49],
       [0.63, 0.37],
       [0.62, 0.38],
       [0.41, 0.59],
       [0.51, 0.49],
       [0.54, 0.46],
       [0.66, 0.34],
       [0.57, 0.43],
       [0.6 , 0.4 ],
       [0.55, 0.45],
       [0.32, 0.68],
       [0.71, 0.29],
       [0.56, 0.44],
       [0.4 , 0.6 ],
       [0.53, 0.47],
       [0.66, 0.34],
       [0.43, 0.57],
       [0.47, 0.53],
       [0.65, 0.35],
       [0.49, 0.51],
       [0.42, 0.58],
       [0.28, 0.72],
       [0.46, 0.54],
       [0.61, 0.39],
       [0.47, 0.53],
       [0.43, 0.57],
       [0.58, 0.42],
       [0.55, 0.45],
       [0.67, 0.33],
       [0.41, 0.59],
       [0.37,